In [ ]:
import pandas as pd
import sqlalchemy
import requests
import datetime
import numpy as np
import datetime as dt

import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout

from keras.optimizers import SGD
from keras.optimizers import RMSprop
from tensorflow.keras.losses import Huber
from keras.optimizers import Nadam
from sklearn.metrics import mean_absolute_percentage_error

In [ ]:
query='''            WITH table1 as 
(
SELECT  DATEADD(WEEK,DATEDIFF(WEEK,0, th.[TranDate]),0) AS DateWeek,p.[Category],p.[Class],p.[StockCode],
p.[Description],tl.[Quantity]
From [PowerBI].[dbo].[TransactionLine] tl 
LEFT JOIN [PowerBI].[dbo].[Subsidiary] s ON tl.SubsidiaryId = s.[SubsidiaryId] 
LEFT JOIN [PowerBI].[dbo].[TransactionHeader] th ON tl.[TransactionId]=th.[TransactionId]
LEFT JOIN [PowerBI].[dbo].[Product] p ON tl.[ProductId] =p.[ProductId]
LEFT JOIN [PowerBI].[dbo].[ItemInventory] i ON i.[ProductId]= p.ProductId 

WHERE --p.[Status] IN ('Current', 'Promo','Clearance') 
p.[StockCode] = 'BW60057GB'
AND s.Name='Wilton Bradley Limited' 
AND p.[Class] IS NOT NULL 
AND p.[Category] IS NOT NULL 
AND  tl.[GlId] =54
AND tl.TransactionType IN ('Cash Sale', 'Credit Memo','Invoice')

)

SELECT  CAST([DateWeek] AS Date) AS DateWeek,[Category],[Class],[StockCode],[Description], SUM([Quantity]) AS Quantity
FROM table1  
GROUP BY [StockCode], [DateWeek], [Description],[Category],[Class]
ORDER BY [DateWeek] ASC

               '''

In [ ]:
engine=sqlalchemy.create_engine("mssql://@192.168.35.102/PowerBi?driver=SQL Server")
connection=engine.connect()
result=connection.execute(sqlalchemy.text(query))

data=[row for row in result]
connection.close()

df=pd.DataFrame(data)
df["DateWeek"]=pd.to_datetime(df["DateWeek"])
df=df[df["DateWeek"]<=datetime.datetime.now()]
df.head()


In [ ]:
import json

last_train_date_file = 'S:\sales office\Vishnu\Forcast for inventory managment\LSTM_Models\Bestway\lastet_train_date\last_train_date.json'

def get_last_train_date():
    try:
        with open(last_train_date_file, 'r') as file:
            data = json.load(file)
            return pd.to_datetime(data['last_train_date'])
    except FileNotFoundError:
        print("Last train date file not found.")
        return None

def update_last_train_date(date):
    with open(last_train_date_file, 'w') as file:
        json.dump({'last_train_date': str(date)}, file)


# Assuming df is your DataFrame
latest_date = df["DateWeek"].max()

# Display the last training date
print("Last training date:", get_last_train_date())

# Update the last training date with the latest date from your DataFrame
update_last_train_date(latest_date)

In [ ]:
# getting covid data
df_covid=pd.read_csv("S:/sales office/Vishnu/Forcast for inventory managment/covid-data.csv")
# filtering onlu uk covid data
df_covid[df_covid["location"]=="United Kingdom"][["date","new_cases"]].sort_values(by="date")

df_cov=df_covid[df_covid["location"]=="United Kingdom"][["date","new_cases"]]
df_cov["date"]=pd.to_datetime(df_cov["date"], format="%d/%m/%Y")
df_cov.set_index("date", inplace=True)
pd.set_option("display.max_columns", None)
df_cov=df_cov.resample('W-Mon').sum()
df_cov.reset_index(inplace=True)
pd.set_option('display.max_rows', None)

initial =pd.Timestamp("2020-03-30")
last=pd.Timestamp("2021-04-30")

df_cov["cases"]= df_cov["date"].apply(lambda x: 1 if initial <=x <=last else 0)

df_cov.head()

In [ ]:
#Creating lags
def create_lag(df):
    number_of_lags=1
    '''
    for lag in range(1,number_of_lags+1):
        df[f"Qty_lag{lag}"]=df["Quantity"].shift(lag)
    '''
        
    for lag in range(1, number_of_lags+1):
        df[f"Qty_diff_lag{lag}"]=df["Quantity"].shift(lag)
        
    return df


In [ ]:
#Creating a week list starting from 2018-12-31
start_date = dt.datetime(2018, 12, 31)
week_dates = []
today= dt.datetime.now()

while start_date<= today:
    week_dates.append(start_date.strftime("%Y-%m-%d"))
    start_date+=dt.timedelta(days=7)

week_dates=pd.DataFrame(week_dates, columns=["Week"])
week_dates["Week"]=pd.to_datetime(week_dates["Week"])
week_dates.head()

In [ ]:
# Remove outliers
def remove_outliers(df, col):
    q1=df[col].quantile(0.25)
    q3=df[col].quantile(0.75)
    
    iqr=q3-q1
    
    lb=q1-1.5*iqr
    ub=q3+1.5*iqr
    
    df1=df[(df[col]<=ub ) &(df[col]>=lb) ]
    
    return df1

def cap_outliers(df, col, cap=95):
    ul=df[col].quantile(cap/100)
    ll=df[col].quantile((100-cap)/100)
    
    df[col]=np.where(df[col]>ul, ul, df[col])
    df[col]=np.where(df[col]<ll, ll, df[col] )
    
    return df

In [ ]:
from tqdm.auto import tqdm
import os
stock_predictions={}

total_stocks=len(df_bestway["StockCode"].unique())
processed_stock=0

model_dir="S:\sales office\Vishnu\Forcast for inventory managment\LSTM_Models\Bestway"
os.makedirs(model_dir, exist_ok=True)

for code in tqdm(df_bestway["StockCode"].unique()):
    
    processed_stock+=1
    percent_complete=(processed_stock/total_stocks)*100
    
    print("/n/nStokCode : ",code)
    print("Processed Stocks : ",processed_stock)
    print("Percent Completed : ", percent_complete)
    single_stock_data=df_bestway[df_bestway["StockCode"].isin([code])].drop(columns=["Category","Class","Description"])

    single_stock_data["DateWeek"]=pd.to_datetime(single_stock_data["DateWeek"])

    single_stock_data=single_stock_data.merge(week_dates, left_on="DateWeek", right_on="Week", how="right")
    single_stock_data["StockCode"].ffill(axis=0, inplace=True)
    single_stock_data=single_stock_data[single_stock_data["StockCode"].notna()]
    single_stock_data.drop("DateWeek",axis=1, inplace=True)
    single_stock_data["Quantity"].fillna(0,inplace=True)
    single_stock_data=single_stock_data.merge(df_cov, how="left", left_on="Week", right_on="date")
    single_stock_data.drop(["date","date","new_cases"],axis=1, inplace=True)
    single_stock_data["cases"].fillna(0, inplace=True)
    single_stock_data.head()
    
    values=single_stock_data[["Quantity","cases"]]

    # Normalize the data
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_values = scaler.fit_transform(values)

    #Create Sequence
    n_input =7 # for example, predicting using 8 weeks 
    n_features = 2  # we are now considering two features
    samples = len(scaled_values) - n_input

    inputs, outputs = [], []
    for i in range(samples):
        input_data = scaled_values[i:(i + n_input)]
        output_data = scaled_values[i + n_input, 0]  # We predict only the 'Quantity' column
        inputs.append(input_data)
        outputs.append(output_data)


    inputs_array = np.array(inputs).reshape(len(inputs), n_input, n_features)
    outputs_array = np.array(outputs)

    #Define the percentage of data you want for testing
    test_pct=0.20

    #Calculate the number of sample for testing
    #num_test_samples=int(inputs_array.shape[0]*test_pct)
    num_test_samples=26

    # Split the data into training and testing sets
    inputs_train=inputs_array[:-num_test_samples]
    outputs_train=outputs_array[:-num_test_samples]
    inputs_test= inputs_array[-num_test_samples:]
    outputs_test=outputs_array[-num_test_samples:]


    #LSTM Model
    model=Sequential()
    model.add(LSTM(1000, activation=PReLU(), input_shape=(n_input, n_features), return_sequences=False))
    #model.add(Dropout(0.3))
    #model.add(LSTM(200, activation="leaky_relu", return_sequences=True))
    #model.add(Dropout(0.2))
    #model.add(LSTM(100, activation="leaky_relu", return_sequences=True))
    #model.add(LSTM(200, activation="leaky_relu", return_sequences=True))
    #model.add(LSTM(50, activation="leaky_relu", return_sequences=False))
    #model.add(LSTM(30, activation="relu", return_sequences=False))

    model.add(Dense(1, activation="linear"))


    optimizer=SGD(lr=0.001, momentum=0.9)
    #optimizer = Nadam(learning_rate=0.01)
    #optimizer = RMSprop(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss="mae")

    # automatically stop training when the model's performance on the validation set starts to degrade
    #early_stopping=EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


    # Train the model
    history = model.fit(inputs_train, outputs_train, epochs=3000, validation_split=0.05, verbose=1)#callbacks=[early_stopping])
    
    model_filename=f"LSTM_{code}.h5"
    model.save(os.path.join(model_dir, model_filename))
    print(f"Model for StockCode {code} saved successfully")
    